In [6]:
# 텐서플로우를 먼저 설치해야 한다
# !pip install keras

In [2]:
import keras
keras.__version__

'2.1.5'

### 신경망 첫걸음 (2.1절)
- Keras 모듈을 사용한다
- 그레이스케일의 28*28 크기의 손글씨 숫자 인식 프로그램을 작성한다
- 1980년대에 NIST에서 만든 MNIST 파일을 사용한다.
- 케라스 패키지에서 Numpy 어레이로 제공한다
- 6만개의 훈련 데이터 10000개의 테스트 데이터제공


In [3]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 41s 4us/step


- 위에 보면 4가지 종류의 데이터셋으로 구성된다
- 레이블은 0~9의 숫자이다.

In [4]:
train_images.shape

(60000, 28, 28)

In [27]:
len(train_labels)

60000

In [28]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [29]:
test_images.shape

(10000, 28, 28)

In [30]:
len(test_labels)

10000

In [40]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

- 신경망을 하나 만들고 train_images와 train_labels 데이터로 학습을 시키겠다.
- 학습시킨 모델에 대해 test_images 데이터를 하지고 테스트를 하겠다.

In [41]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

- 신경망의 핵심 구성은 바로 계층(layer)이다. 여기서 데이터를 처리(필터링)한다.
- 계층은 데이터로부터 어떤 표현(representation)을 추출하는 것이라고 할 수 있다.
- 신경망은 이러한 계층을 여러개 사용하여 데이터로부터 점차 의미있는 정보를 추출(정제)한다.
- 딥러닝이란 이러한 계층이 여러개 있는 구조를 말한다.

- 위의 예에서는 2개의 "Dense" 타입의 계층을 사용했다. 여기서 dense의 의미는 fully-connected를 의미한다.
- 첫번째 계층은 활성함수로 relu를 사용했고, 두번째 계층은 softmax를 사용했다.
- 두번째 계층은 출력이 10개이며 이들이 각각 0~9를 예상하는 확률을 나타나게 모델링을 한다. 10개 출력의 합은 1이 된다. (softmax 함수의 특징이다)

### 추가 작업
- 손실함수 정의: 훈련 데이터로 모델을 만들어 나갈때 이 값을 최소화 하는 방향으로 모델을 만들어간다
- 최적화 알고리즘: 손실함수를 줄이기 위해서 선택한 알고리즘
- 최종 성능 평가: 모델의 최종 성능을 측정하는 평가 척도를 말한다. 여기서는 정확도(accuracy)를 사용하겠다.

In [42]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

- 전처리 작업: 훈련을 시작하기 전에 데이터를 가공할 필요가 있다.
- 현재 가지고 있는 데이터는 (60000, 28, 28) 형태로서 각 항목은 [0, 255] 사이의 정수 값을 갖는다
- 이를 (60000, 28 * 28) 구조의 (0, 1) 사이의 소수로 변환하겠다.
- 테스트 데이터에 대해서도 동일한 변화을 해두어야 한다.

In [43]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

We also need to categorically encode the labels, a step which we explain in chapter 3:

- 레이블의 값이 현재는 0~9의 정수를 나타내는데 이를 범주형(categorical) 타입으로 바꾸어야 한다. (수치를 예측하는 것이 아니라 문자 분류 문제이므로)

In [44]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

We are now ready to train our network, which in Keras is done via a call to the `fit` method of the network: 
we "fit" the model to its training data.

- 이제 fit 함수를 사용하여 모델을 주어진 훈련데이터에 맞추는(fitting) 작업을 수행한다

In [45]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 2s 38us/step - loss: 0.2578 - acc: 0.9271

- 손실 값과 정확도(평가지표) 두가지 정보를 보여준다
- 정확도 98.9%나 되는 결과를 얻어다. 
- 이 모델을 사용하여 테스트 데이터에 적용해 보겠다.

In [46]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

10000/10000 [==============================] - 0s 24us/step


In [47]:
print('test_acc:', test_acc)

test_acc: 0.9785


- 테스트 데이터의 성능이 훈련 데이터의 결과보다 많이 낮다. 이를 과적합되었다는 것을 의미한다.
- 여기서는 간단히 데이터분석 전체 과정을 살펴보았다.
- 텐서의 개념,경사하강법 등을 배우겠다.